# 8.2.5 Alternative Gauss Jordon Algorithm

In this notebook, you will implement the alternative Gauss Jordan algorithm that overwrites $ A $ in one sweep with the identity matrix and $ B $ with the inverse of the original matrix $ A $.

<font color=red> Be sure to make a copy!!!! </font>

<h2> First, let's create a matrix $ A $ and set $ B $ to the identity. </h2>

In [4]:
include("../flame.jl")
include("../laff/laff.jl")

# `LinearAlgebra` us to use LinearAlgebra.I to create an identity matrix
using LinearAlgebra

In [16]:
L = [ 1  0  0  0
     -2  1  0  0
      1 -3  1  0
      2  3 -1  1. ]

U = [ 2 -1  3 -2
      0 -2  1 -1
      0  0  1  2
      0  0  0  3 ]

A = L * U
Aold = copy(A)
println("A = ")
A

A = 


4×4 Array{Float64,2}:
  2.0  -1.0   3.0  -2.0
 -4.0   0.0  -5.0   3.0
  2.0   5.0   1.0   3.0
  4.0  -8.0   8.0  -6.0

In [17]:
B = Matrix{Float64}(I, 4, 4)
println(" B = ")
B

 B = 


4×4 Array{Float64,2}:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0

In [18]:
# # We can create identity matrices in a few ways!
# # Try these out: 
# B = Matrix{Int}(I, 4, 4)
# B = Diagonal(ones(4))

# # What do you get when you try the following?
# B = Matrix(I, 4, 4)

<h2> Implement the alternative Gauss-Jordan algorithm from 8.2.5 </h2>

Here is the algorithm:

<img src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/8_2_5_Algorithm.png" alt="Alternative Gauss-Jordan algorithm" width=100%>
    
<font color=red> Important: if you make a mistake, rerun ALL cells above the cell in which you were working, and then the one where you are working. </font>

Create the routine
<code> GJ_Inverse_alt! </code>
with the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> for the algorithm

In [19]:
include("../flame.jl")
include("../laff/laff.jl")

function GJ_Inverse_alt!(A, B)

    ATL, ATR, 
    ABL, ABR  = flame.part_2x2(A, 
                               0, 0, "TL")

    BTL, BTR, 
    BBL, BBR  = flame.part_2x2(B, 
                               0, 0, "TL")

    while size(ATL, 1) < size(A, 1)

        A00,  a01,     A02,  
        a10t, alpha11, a12t, 
        A20,  a21,     A22   = flame.repart_2x2_to_3x3(ATL, ATR, 
                                                       ABL, ABR, 
                                                       1, 1, "BR")

        B00,  b01,    B02,  
        b10t, beta11, b12t, 
        B20,  b21,    B22   = flame.repart_2x2_to_3x3(BTL, BTR, 
                                                      BBL, BBR, 
                                                      1, 1, "BR")

        #------------------------------------------------------------#

        # a01 := a01 / alpha11
        # a21 := a21 / alpha11
        laff.invscal!( alpha11, a01 )
        laff.invscal!( alpha11, a21 )
        
        # A02 := A02 - a01 * a12t
        laff.ger!( -1.0, a01, a12t, A02 )
        # A22 := A22 - a21 * a12t
        laff.ger!( -1.0, a21, a12t, A22 )
        
        # B00 := B00 - a01 * b01t
        laff.ger!( -1.0, a01, b10t, B00 )
        # B20 := B20 - a21 * b01t
        laff.ger!( -1.0, a21, b10t, B20 )
        
        # b01 := - a01 (= - u01 in the discussion)
        laff.copy!( a01, b01 )
        laff.scal!( -1.0, b01 )
        # b21 := - a21 (= - l21 in the discussion)
        laff.copy!( a21, b21 )
        laff.scal!( -1.0, b21 )
        
        # a12t:= a21t / alpha11  
        laff.invscal!( alpha11, a12t )
        # b10t:= b10t / alpha11 
        laff.invscal!( alpha11, b10t )
        
        # beta11 := 1.0 / alpha11
        laff.invscal!( alpha11, beta11 )
       
        # a01 = 0  (zero vector)
        # alpha11 = 1
        # a21 = 0  (zero vector)
        laff.zerov!( a01 )
        laff.onev!( alpha11 )
        laff.zerov!( a21 )

        #------------------------------------------------------------#

        ATL, ATR, 
        ABL, ABR  = flame.cont_with_3x3_to_2x2(A00,  a01,     A02,  
                                               a10t, alpha11, a12t, 
                                               A20,  a21,     A22,  
                                               "TL")

        BTL, BTR, 
        BBL, BBR  = flame.cont_with_3x3_to_2x2(B00,  b01,    B02,  
                                               b10t, beta11, b12t, 
                                               B20,  b21,    B22,  
                                               "TL")

    end

    flame.merge_2x2!(ATL, ATR, 
                     ABL, ABR, A)

    flame.merge_2x2!(BTL, BTR, 
                     BBL, BBR, B)

end

GJ_Inverse_alt! (generic function with 1 method)

<h3> Test the routine </h3>

<font color=red> Important: if you make a mistake, rerun ALL cells above the cell in which you were working, and then the one where you are working. </font>

In [20]:
GJ_Inverse_alt!( A, B )

In [21]:
A

4×4 Array{Float64,2}:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0

In [22]:
B

4×4 Array{Float64,2}:
 -9.5  -4.0  -0.166667   1.08333 
  3.0   1.0   0.0       -0.5     
  7.0   3.0   0.333333  -0.666667
 -1.0   0.0   0.333333   0.333333

Matrix $ A $ should now be an identity matrix and $ B $ should no longer be an identity matrix.

Check if $ B $ now equals (approximately) the inverse of the original matrix $ A $:

In [23]:
Aold * B

4×4 Array{Float64,2}:
 1.0  0.0  0.0           0.0        
 0.0  1.0  5.55112e-17   5.55112e-17
 0.0  0.0  1.0          -5.55112e-17
 0.0  0.0  1.11022e-16   1.0        